# Projeto 1: Fases de Modeling & Evaluation (CRISP-DM)

**Objetivo:** Utilizar os dados preparados para treinar modelos de Machine Learning (Regressão) capazes de prever o Nível de Stress e a Qualidade do Sono.

**Etapas:**
1.  **Setup dos Dados:** (Recapitulação rápida do Data Preparation).
2.  **Split dos Dados:** Divisão em Treino (80%) e Teste (20%).
3.  **Modelação (Target 1: Stress):** Treino de Regressão Linear e Random Forest.
4.  **Avaliação (Target 1: Stress):** Análise de métricas (RMSE, R²) e Resíduos.
5.  **Modelação e Avaliação (Target 2: Sono):** Repetição do processo para a Qualidade do Sono.
6.  **Feature Importance:** Identificar quais variáveis mais influenciam as previsões.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Configurar estilo
sns.set_theme(style="whitegrid")
%matplotlib inline

## 1. Setup dos Dados (Recap do Data Preparation)

Para garantir que este notebook é executável de forma independente, aplicamos as transformações definidas no `Data_Preparation_v2.ipynb`.

In [ ]:
# 1. Carregar
try:
    df = pd.read_csv('../assets/ScreenTime vs MentalWellness.csv')
except FileNotFoundError:
    print("Erro: Ficheiro csv não encontrado.")

df_prep = df.copy()
df_prep = df_prep.drop('user_id', axis=1)

# 2. Feature Engineering
df_prep['other_screen_hours'] = df_prep['screen_time_hours'] - (df_prep['work_screen_hours'] + df_prep['leisure_screen_hours'])
df_prep['other_screen_hours'] = df_prep['other_screen_hours'].clip(lower=0)

# 3. Capping de Outliers (1% - 99%)
cols_to_cap = ['age', 'screen_time_hours', 'work_screen_hours', 'leisure_screen_hours', 
               'sleep_hours', 'productivity_0_100', 'exercise_minutes_per_week', 
               'social_hours_per_week', 'other_screen_hours']

for col in cols_to_cap:
    lower = df_prep[col].quantile(0.01)
    upper = df_prep[col].quantile(0.99)
    df_prep[col] = df_prep[col].clip(lower=lower, upper=upper)

# 4. Encoding e Scaling
df_prep = pd.get_dummies(df_prep, columns=['gender', 'occupation', 'work_mode'], drop_first=True)

target_cols = ['stress_level_0_10', 'sleep_quality_1_5', 'mental_wellness_index_0_100']
features_to_scale = [col for col in df_prep.columns if col not in target_cols]
if 'screen_time_hours' in features_to_scale: features_to_scale.remove('screen_time_hours')

scaler = StandardScaler()
df_final = df_prep.copy()
df_final[features_to_scale] = scaler.fit_transform(df_final[features_to_scale])

# 5. Definir X e y globais
X = df_final[features_to_scale]
y_stress = df_final['stress_level_0_10']
y_sleep = df_final['sleep_quality_1_5']

print("Dados preparados e carregados com sucesso!")
print(f"Features (X): {X.shape}")

## 2. Split dos Dados (Treino e Teste)

Os dados são divididos em:
- **Treino (80%):** Usado para o modelo aprender os padrões.
- **Teste (20%):** Usado para avaliar se o modelo consegue prever dados que nunca viu antes.

Faremos dois splits diferentes, um para cada variável alvo.

In [ ]:
# Split para Stress
X_train_stress, X_test_stress, y_train_stress, y_test_stress = train_test_split(
    X, y_stress, test_size=0.2, random_state=42
)

# Split para Sleep Quality
X_train_sleep, X_test_sleep, y_train_sleep, y_test_sleep = train_test_split(
    X, y_sleep, test_size=0.2, random_state=42
)

print(f"Tamanho do Treino: {X_train_stress.shape[0]} amostras")
print(f"Tamanho do Teste: {X_test_stress.shape[0]} amostras")

## 3. Definição da Função de Avaliação

**Métricas Utilizadas:**
- **MAE (Mean Absolute Error):** Erro médio absoluto.
- **RMSE (Root Mean Squared Error):** Penaliza erros maiores. Útil para ver se o modelo falha muito.
- **R² (R-Squared):** Explica a percentagem da variância. (1.0 = Perfeito, 0.0 = Modelo aleatório).

In [ ]:
def avaliar_modelo(modelo, X_train, X_test, y_train, y_test, nome_modelo="Modelo"):
    # 1. Treinar
    modelo.fit(X_train, y_train)
    
    # 2. Prever
    y_pred = modelo.predict(X_test)
    
    # 3. Métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"--- {nome_modelo} ---")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²:   {r2:.4f}")
    
    return y_test, y_pred, modelo

## 4. Modelação - Cenário A: Prever Nível de Stress (0-10)

1.  **Regressão Linear:** 
2.  **Random Forest Regressor:**

In [ ]:
print("=== AVALIAÇÃO: PREVISÃO DE STRESS ===\n")

# Modelo 1: Regressão Linear
lr_stress = LinearRegression()
y_test_s, y_pred_lr_s, model_lr_s = avaliar_modelo(lr_stress, X_train_stress, X_test_stress, y_train_stress, y_test_stress, "Regressão Linear (Stress)")

print("\n")

# Modelo 2: Random Forest
rf_stress = RandomForestRegressor(n_estimators=100, random_state=42)
y_test_s, y_pred_rf_s, model_rf_s = avaliar_modelo(rf_stress, X_train_stress, X_test_stress, y_train_stress, y_test_stress, "Random Forest (Stress)")

### Visualização dos Resultados (Stress)


In [ ]:
plt.figure(figsize=(14, 5))

# Scatter plot: Real vs Previsto (Random Forest)
plt.subplot(1, 2, 1)
plt.scatter(y_test_s, y_pred_rf_s, alpha=0.5, color='blue')
plt.plot([0, 10], [0, 10], color='red', linestyle='--') # Linha perfeita
plt.xlabel("Stress Real")
plt.ylabel("Stress Previsto")
plt.title("Real vs Previsto (Random Forest)")

# Resíduos (Erro por previsão)
plt.subplot(1, 2, 2)
residuos = y_test_s - y_pred_rf_s
sns.histplot(residuos, kde=True, color='purple')
plt.title("Distribuição dos Erros (Resíduos)")
plt.xlabel("Erro (Real - Previsto)")

plt.tight_layout()
plt.show()

**Análise (Stress):**
- Se o **R²** for alto (perto de 1.0), conseguimos explicar bem o stress com os dados que temos.
- Se o gráfico de dispersão seguir a linha vermelha, o modelo está correto.

## 5. Modelação - Cenário B: Prever Qualidade do Sono (1-5)

Repetimos o processo para a variável `sleep_quality_1_5`.

In [ ]:
print("=== AVALIAÇÃO: PREVISÃO DE QUALIDADE DO SONO ===\n")

# Modelo 1: Regressão Linear
lr_sleep = LinearRegression()
y_test_sl, y_pred_lr_sl, model_lr_sl = avaliar_modelo(lr_sleep, X_train_sleep, X_test_sleep, y_train_sleep, y_test_sleep, "Regressão Linear (Sono)")

print("\n")

# Modelo 2: Random Forest
rf_sleep = RandomForestRegressor(n_estimators=100, random_state=42)
y_test_sl, y_pred_rf_sl, model_rf_sl = avaliar_modelo(rf_sleep, X_train_sleep, X_test_sleep, y_train_sleep, y_test_sleep, "Random Forest (Sono)")

## 6. Feature Importance (O que influencia mais?)

Uma parte crucial do *Business Understanding* é explicar o "porquê". O Random Forest permite-nos ver quais variáveis foram mais importantes para a decisão.

In [ ]:
# Extrair importância das features do modelo de Stress
importances = model_rf_s.feature_importances_
feature_names = X.columns

# Criar DataFrame para visualização
feature_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_imp_df = feature_imp_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=feature_imp_df, palette='viridis')
plt.title("Importância das Variáveis na Previsão do Stress")
plt.show()

## 7. Conclusões Finais

**1. Desempenho dos Modelos:**

**2. Drivers de Stress:**
